In [ ]:
#!pip install geopandas

In [1]:
import pandas as pd
import geopandas as gpd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
!pip install --upgrade botocore==1.23.26  #Sur colab, sinon bug
!pip install --upgrade urllib3==1.22.0 #Sur colab, sinon bug
!pip install py7zr #Sur colab, sinon bug
!pip install s3fs #Sur colab, sinon bug
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .
%cd ..

In [2]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [3]:
#Base adresses -> coordonnées GPS
df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)

/tmp/ipykernel_8012/2004415946.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)
/tmp/ipykernel_8012/2004415946.py:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_adresses= pd.read_csv("https://adresse.data.gouv.fr/data/ban/adresses/latest/csv/adresses-38.csv.gz", compression='gzip', sep=";", error_bad_lines=False)


In [4]:
#Coordonnnées GPS ->coordonnées Geopandas
gpd_adresses= gpd.GeoDataFrame(df_adresses, geometry=gpd.points_from_xy(df_adresses.lon, df_adresses.lat))

In [5]:
#Base risque d'inondations niveau national
df_risque_inond=gpd.read_file("DataSets/tri_2020_sig_di/n_carte_inond_s.shp") 

In [6]:
#Coordonnées de l'habitation (en géometry) ->risque d'inondation
#gpd_adresses : tables geopandas qui contient les adresses du département
def adresse_to_geometry(dep,num,nom_voie,ville,code_postal):
  return(gpd_adresses[(gpd_adresses["numero"] == num) & (gpd_adresses["nom_commune"] == ville) & (gpd_adresses["nom_voie"] == nom_voie)]["geometry"])


In [7]:
#Fonction geometry ->risque 
def risque(geometry):
  risque=[]
  for k in range(len(df_risque_inond["geometry"])):
    if df_risque_inond["geometry"][k].contains(geometry):
      risque.append(df_risque_inond["scenario"][k])
  
  return(risque)

In [8]:
#Fonction finale
def adresse_to_risque(dep,num,nom_voie,ville,code_postal):
  return(risque(adresse_to_geometry(dep,num,nom_voie,ville,code_postal)))

